In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter

## Stacking images

In [ ]:
data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_CLUSTERS)

In [ ]:
reduced = True
aligned = True
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

In [ ]:
subdata_dir = os.path.join(data_dir, os.listdir(data_dir)[0])
if reduced:
    subdata_dir = os.path.join(subdata_dir, "REDUCED")
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
if os.path.isdir(subdata_dir):
    list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
    list_fits = sorted(list_fits)

In [ ]:
list_fits

In [ ]:
# !swarp $SCIMAGESC2PU/Photometry/Clusters/2021-10-06_OMICRON_F3p17_OPF_QHY600Ma_hyades/REDUCED/aligned/*SDSSi*.fits -c defaults.swarp -IMAGEOUT_NAME coadd_2021-10-06_OMICRON_F3p17_OPF_QHY600Ma_hyades_SDSSip.fits -CENTER_TYPE ALL

In [ ]:
from tqdm import tqdm

SDSSi_imgs = [os.path.join(subdata_dir, im) for im in list_fits if "SDSSi" in im]
with fits.open(SDSSi_imgs[0]) as hdul:
    ref_hdu_i = hdul[0].copy()
    SDSSi_stack = np.empty((len(SDSSi_imgs), *hdul[0].data.shape))
for loc, fits_img in enumerate(tqdm(SDSSi_imgs)):
    with fits.open(fits_img) as hdul:
        img_data = hdul[0].data
    SDSSi_stack[loc, :, :] = img_data
    # mean, med, sigma = sigma_clipped_stats(img_data, sigma=3)
    # plt.imshow(img_data, cmap='gray', vmin=med-5*sigma, vmax=med+5*sigma)
    # plt.colorbar()
    # plt.show()

In [ ]:
coadd_i = np.mean(SDSSi_stack, axis=0)
mean, med, sigma = sigma_clipped_stats(coadd_i, sigma=3)

plt.imshow(coadd_i, cmap="gray", vmin=med - 5 * sigma, vmax=med + 5 * sigma)
plt.colorbar()

In [ ]:
ref_hdu_i.data = coadd_i
# ref_hdu_i.writeto(f'coadd_{os.listdir(data_dir)[0]}_SDSSip.fits', overwrite=True)

In [ ]:
with fits.open("coadd_2021-10-06_OMICRON_F3p17_OPF_QHY600Ma_hyades_SDSSip.fits") as stack_ip:
    hdr = stack_ip[0].header
    data = stack_ip[0].data
mean, med, sigma = sigma_clipped_stats(data, sigma=3)
plt.imshow(data, cmap="gray", vmin=med - 5 * sigma, vmax=med + 5 * sigma)
plt.colorbar()

__If not done, plate-solve within AIJ.__

In [ ]:
SDSSg_imgs = [os.path.join(subdata_dir, im) for im in list_fits if "SDSSg" in im]

with fits.open(SDSSg_imgs[0]) as hdul:
    ref_hdu_g = hdul[0].copy()
    SDSSg_stack = np.empty((len(SDSSg_imgs), *hdul[0].data.shape))

for loc, fits_img in enumerate(tqdm(SDSSg_imgs)):
    with fits.open(fits_img) as hdul:
        img_data = hdul[0].data
    SDSSg_stack[loc, :, :] = img_data

coadd_g = np.mean(SDSSg_stack, axis=0)
ref_hdu_g.data = coadd_g
# ref_hdu_g.writeto(f'coadd_{os.listdir(data_dir)[0]}_SDSSgp.fits', overwrite=True)

In [ ]:
SDSSr_imgs = [os.path.join(subdata_dir, im) for im in list_fits if "SDSSr" in im]

with fits.open(SDSSr_imgs[0]) as hdul:
    ref_hdu_r = hdul[0].copy()
    SDSSr_stack = np.empty((len(SDSSr_imgs), *hdul[0].data.shape))

for loc, fits_img in enumerate(tqdm(SDSSr_imgs)):
    with fits.open(fits_img) as hdul:
        img_data = hdul[0].data
    SDSSr_stack[loc, :, :] = img_data

coadd_r = np.mean(SDSSr_stack, axis=0)
ref_hdu_r.data = coadd_r
# ref_hdu_r.writeto(f'coadd_{os.listdir(data_dir)[0]}_SDSSrp.fits', overwrite=True)

## References in PANSTARRS

In [ ]:
from du_astro_utils import query_panstarrs

In [ ]:
ref_ps = query_panstarrs(f"coadd_{os.listdir(data_dir)[0]}_SDSSgp.fits")

In [ ]:
ref_ps

In [ ]:
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.coordinates import Angle, SkyCoord

coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])

## Photometry

In [ ]:
from astropy.table import Table, vstack

from astropy.time import Time
import warnings
from astropy.utils.exceptions import AstropyWarning
from astropy.coordinates.name_resolve import NameResolveError

use_sextractor = True

red_sci_image = f"coadd_{os.listdir(data_dir)[0]}_SDSSgp.fits"
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("DATE-OBS"), format="isot")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME tmp_sources.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET"
        os.system(sex_cmd)
        cat_tab = Table.read("tmp_sources.cat")
        cat_tab.rename_column("X_IMAGE", "xcentroid")
        cat_tab.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab["FWHM_IMAGE"], sigma=3)
        source_coords = SkyCoord(ra=cat_tab["ALPHA_J2000"], dec=cat_tab["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources)
        except RuntimeError:
            fwhm = 10
        cat_tab = photometry.apert_photometry(red_sci_image, sources, fwhm)
        source_coords = SkyCoord.from_pixel(cat_tab["xcenter"], cat_tab["xcenter"], wcs)

In [ ]:
cat_tab

In [ ]:
import astropy.units as u

xm_id, xm_ang_distance, _ = source_coords.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
print(hdu.header.get("PIXSCALX") * fwhm)
max_sep = hdu.header.get("PIXSCALX") * fwhm * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = source_coords[sep_constraint]
catalog_matches = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches = coord_panstarrs[xm_id[sep_constraint]]

In [ ]:
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag = -2.5 * np.log10(cat_tab[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag = ref_ps["gmag"][xm_id[sep_constraint]]

    ins_err = ins_mag - -2.5 * np.log10((cat_tab[sep_constraint]["aper_sum_bkgsub"] + cat_tab[sep_constraint]["noise"]) / exptime)
    cat_err = ref_ps["e_gmag"][xm_id[sep_constraint]]

    cat_tab["ins_mag"] = 0
    cat_tab["ins_mag"][sep_constraint] = ins_mag
else:
    cat_mag = ref_ps["gmag"][xm_id[sep_constraint]]
    cat_err = ref_ps["e_gmag"][xm_id[sep_constraint]]
    ins_mag = cat_tab[sep_constraint]["MAG_AUTO"]
    ins_err = cat_tab[sep_constraint]["MAGERR_AUTO"]

In [ ]:
plt.scatter(ins_mag, cat_mag)

In [ ]:
plt.scatter(cat_mag, ins_mag - cat_mag)

In [ ]:
from sklearn import linear_model

# Selection from magnitude range
mag_min, mag_max = 12, 19
cond = (cat_mag > mag_min) & (cat_mag < mag_max) & (~cat_mag.mask) & (~np.isnan(ins_mag))

# Create two mock arrays for linear regression
X = ins_mag[cond].reshape(-1, 1)
y = cat_mag[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

In [ ]:
# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab["AB_MAG"] = 0.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab["aper_sum_bkgsub"] > 0)
    cat_tab["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab["FLUX_AUTO"] > 0)
    cat_tab["AB_MAG"][positive] = ransac.predict(cat_tab[positive]["MAG_AUTO"].data.reshape(-1, 1)).flatten()
cat_tab

In [ ]:
write_res = False

In [ ]:
red_sci_image = f"coadd_{os.listdir(data_dir)[0]}_SDSSrp.fits"
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("DATE-OBS"), format="isot")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME tmp_sources.cat -CATALOG_TYPE FITS_1.0 -GAIN 0.932 -VERBOSE_TYPE QUIET"
        os.system(sex_cmd)
        catR_tab = Table.read("tmp_sources.cat")
        catR_tab.rename_column("X_IMAGE", "xcentroid")
        catR_tab.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(catR_tab["FWHM_IMAGE"], sigma=3)
        sourceR_coords = SkyCoord(ra=catR_tab["ALPHA_J2000"], dec=catR_tab["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sourcesR = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sourcesR)
        except RuntimeError:
            fwhm = 10
        catR_tab = photometry.apert_photometry(red_sci_image, sourcesR, fwhm)
        sourceR_coords = SkyCoord.from_pixel(catR_tab["xcenter"], cat_tab["xcenter"], wcs)

In [ ]:
xm_id, xm_ang_distance, _ = sourceR_coords.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
print(hdu.header.get("PIXSCALX") * fwhm)
max_sep = hdu.header.get("PIXSCALX") * fwhm * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = sourceR_coords[sep_constraint]
catalog_matches = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches = coord_panstarrs[xm_id[sep_constraint]]

In [ ]:
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag = -2.5 * np.log10(catR_tab[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag = ref_ps["rmag"][xm_id[sep_constraint]]

    ins_err = ins_mag - -2.5 * np.log10((catR_tab[sep_constraint]["aper_sum_bkgsub"] + catR_tab[sep_constraint]["noise"]) / exptime)
    cat_err = ref_ps["e_rmag"][xm_id[sep_constraint]]

    catR_tab["ins_mag"] = 0
    catR_tab["ins_mag"][sep_constraint] = ins_mag
else:
    cat_mag = ref_ps["rmag"][xm_id[sep_constraint]]
    cat_err = ref_ps["e_rmag"][xm_id[sep_constraint]]
    ins_mag = catR_tab[sep_constraint]["MAG_AUTO"]
    ins_err = catR_tab[sep_constraint]["MAGERR_AUTO"]
plt.scatter(ins_mag, cat_mag)
plt.xlim(-20, 5)

In [ ]:
# Selection from magnitude range
mag_min, mag_max = 12, 19
cond = (cat_mag > mag_min) & (cat_mag < mag_max) & (~cat_mag.mask) & (~np.isnan(ins_mag))

# Create two mock arrays for linear regression
X = ins_mag[cond].reshape(-1, 1)
y = cat_mag[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

In [ ]:
# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
catR_tab["AB_MAG"] = 0.0

# Positive values
if not use_sextractor:
    positive = np.where(catR_tab["aper_sum_bkgsub"] > 0)
    catR_tab["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(catR_tab[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab["FLUX_AUTO"] > 0)
    catR_tab["AB_MAG"][positive] = ransac.predict(catR_tab[positive]["MAG_AUTO"].data.reshape(-1, 1)).flatten()
catR_tab

In [ ]:
len(catR_tab[catR_tab["AB_MAG"] > 0])

## HR diagram

In [ ]:
xm_id, xm_ang_distance, _ = sourceR_coords.match_to_catalog_sky(source_coords, nthneighbor=1)
print(hdu.header.get("PIXSCALX") * fwhm)
max_sep = hdu.header.get("PIXSCALX") * fwhm * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = sourceR_coords[sep_constraint]
catalog_matches = cat_tab[xm_id[sep_constraint]]
coord_catalog_matches = source_coords[xm_id[sep_constraint]]

In [ ]:
g_cat = catalog_matches
r_cat = catR_tab[sep_constraint]

sel1 = np.logical_and(g_cat["AB_MAG"] > 0.0, r_cat["AB_MAG"] > 0.0)
sel2 = np.logical_and(g_cat["AB_MAG"] < 99.0, r_cat["AB_MAG"] < 99.0)
sel = np.logical_and(sel1, sel2)
plt.scatter(g_cat[sel]["AB_MAG"] - r_cat[sel]["AB_MAG"], g_cat[sel]["AB_MAG"], marker=".")

In [ ]:
import pandas as pd

iso_SDSS = pd.read_csv("TD_8_AmasStellaires/isochrones/output_SDSSugriz_0.1Gyr-10Gyr.dat", header=0, comment="#", delimiter="\s+", skipinitialspace=True, skipfooter=1)

In [ ]:
iso_SDSS.columns

In [ ]:
g_cat = catalog_matches
r_cat = catR_tab[sep_constraint]

sel1 = np.logical_and(g_cat["AB_MAG"] > 0.0, r_cat["AB_MAG"] > 0.0)
sel2 = np.logical_and(g_cat["AB_MAG"] < 99.0, r_cat["AB_MAG"] < 99.0)
sel = np.logical_and(sel1, sel2)
plt.scatter(g_cat[sel]["AB_MAG"] - r_cat[sel]["AB_MAG"], g_cat[sel]["AB_MAG"], marker=".")

In [ ]:
import seaborn as sns

iso_SDSS["g-r"] = iso_SDSS["gmag"] - iso_SDSS["rmag"]
sns.scatterplot(data=iso_SDSS, x="g-r", y="gmag", hue="logAge")

In [ ]:
f, a = plt.subplots(1, 1)
sns.scatterplot(data=iso_SDSS, x="g-r", y="gmag", hue="logAge", ax=a)
a.scatter(g_cat[sel]["AB_MAG"] - r_cat[sel]["AB_MAG"], g_cat[sel]["AB_MAG"], marker=".", label="Hyades")